In [1]:
%run init_notebook.py

In [2]:
import pandas as pd
import numpy as np

import os

In [3]:
from settings import OUT_MODEL_DIR
from utils.load import load_data, save_file

In [4]:
# directory overview

In [5]:
def get_model_list(path):
    dir_list = os.listdir(OUT_MODEL_DIR)
    dir_ist = [item for item in dir_list if '.' not in item]
    return dir_list

In [6]:
def get_model_dict(path,
                   dict_file_end: str = '_data_dict.pkl'):
    dir_list = os.listdir(path)
    dict_name = [item for item in dir_list if dict_file_end in item]
    
    if len(dict_name) > 1:
        dict_name = dict_name[-1]
    else:
        dict_name = dict_name[0]
    
    file = load_data(file_name=dict_name, file_path=path)
    return file    

In [8]:
class ModelMetrics:
    
    def __init__(self,
                 dict_):
        self._metrics = dict_["model_metrics"]
        
        self.model_type = self._metrics["model_type"]
        self.predicted = dict_["model_features"]["predicted"]
        self.data_len = self._metrics["data_len"]
        self.is_sent = self._metrics["is_sent"]
        self.mse = self._metrics["mse"]
        self.mae = self._metrics["mae"]
        self.r2 = self._metrics["r2"]
        pass

In [9]:
def get_metrics_overview(model_list: list):
    
    model_type = []
    predicted = []
    data_len = []
    is_sent = []
    mse = []
    mae = []
    r2 = []
    col_name = []
    for i, item in enumerate(model_list):
        model_type.append(item.model_type)
        predicted.append(item.predicted)
        data_len.append(item.data_len)
        is_sent.append(item.is_sent)
        mse.append(item.mse)
        mae.append(item.mae)
        r2.append(item.r2)
        col_name.append(f'model_{i}')
    
    df = pd.DataFrame([model_type,
                       predicted,
                       data_len,
                       is_sent,
                       mse,
                       mae,
                       r2],
                     index=["model_type", "predicted", "data_len", "is_sent", "mse", "mae", "r2"])
    
    return df.transpose()
        
        

In [10]:
model_list = []
for item in get_model_list(OUT_MODEL_DIR):
    model_list.append(ModelMetrics(get_model_dict(os.path.join(OUT_MODEL_DIR, item))))
df = get_metrics_overview(model_list)

In [12]:
df.pivot_table(index=["is_sent", "model_type"],
               columns=["predicted"],
               values=["mse"])
               
               

mse                
predicted          sp_close_lead1 sp_close_lead14
is_sent model_type                               
True    lgb              0.000224        0.000582
        ols              0.000241        0.000499